<a href="https://colab.research.google.com/github/Keenandrea/robo-trigger/blob/master/robotrigger_nadam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import glob
import pickle
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint
from music21 import converter, instrument, note, chord

In [11]:
run_on_colab = True
if(run_on_colab):
  from google.colab import drive
  # This will prompt for authorization.
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip install music21;

In [13]:
from google.colab import files
files.upload()

Saving battles.zip to battles.zip


{'battles.zip': b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xe1\xa6jNe\xe69\xf5\x15\x03\x00\x00u\x16\x00\x00"\x00\x00\x00battles/piano-a-shot-of-crisis.mid\xed\x98OO\x13A\x18\xc6_X\x10R\x0e\x1e\xf0\xa0\xc6l\xdaK/\x84\xd8\x10IC\xb3!$\x86d\xb3\xb8\xd9v\xb3\xb6\xd2\x04\xb4f+4J!\x05\x12<\x18\xbb6\x81k\xff\x9c\x89_\x01\xfd\x04\xa2~\r\xbf\x82go\xeb\xb33\xb3\xc3R\xf0B!@\xed\xa1}\xfb\xfc\x9ew\xfa\xcclg\xd3\xc9\x9a\xce\xbaKDwh\x88Fi\xcftjo\xa1\xd6\xc8\x7f122|\x7f\x9c\xfc\x9cr\xe7a\x9d|\xe5\xc1\xd3\xf5\xdafu3\xee\xd4*kk\xe5Z&n\x97w*;\xe5\xedD\x82\xfc\xa1\xf1\xe7\xd5ji\xa3\xec\xc6\xfe\xfd\xd9\x19}E\n\xbe\xda_\x1e\xc6\xfbO\xfaC\xfec\xe2q\xc3\xbf\xc8O\x0c\xc1R\xeev\x85\xd07\xa2\xa3\xb1\x8ft\x14[\xa0f\xce\xa5f\xdeU\xf0b\x9f\xeb9\xa2z\x9e\x1a\t\x01\xee\x85`Rv\xe4)\xe8R\xc3\xda\xb4\xc1\x0b\xee\xfe\x9b\xba\rP\x90@\x95\xda\x81\xb6\x82\x06\x07\xc0\x92@\x95\x9a\xcf\x01\r<\xab(\xa2&E\xc4\x84hP\xbb&\xb9\xaf_\xf9\x9c\x0eVE\x96\n\xd3{\xd24]\xeeZ\xa8&j\x96i\x18\x01\xc8\x02\x9cn\x80a0\xa01\xc3@\xd5\x99\xa11\xaeG\xb8\x

In [14]:
!unzip battles.zip;

Archive:  battles.zip
  inflating: battles/piano-a-shot-of-crisis.mid  
  inflating: battles/piano-battle-five.mid  
  inflating: battles/piano-battle-one.mid  
  inflating: battles/piano-battle-two.mid  
  inflating: battles/piano-bike-chase.mid  
  inflating: battles/piano-boss-battle-one.mid  
  inflating: battles/piano-boss-battle-two.mid  
  inflating: battles/piano-lavos-core-last-battle-three.mid  


In [15]:
notes = []
for i,file in enumerate(glob.glob("battles/*.mid")):
  midi = converter.parse(file)
  print('\r', 'Parsing file ', i, " ",file, end='')
  notes_to_parse = None
  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

 Parsing file  7   battles/piano-boss-battle-one.mid

In [16]:
# Count different possible outputs
n_vocab = (len(set(notes)))
n_vocab

137

In [0]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length] # Size sequence_length
  sequence_out = notes[i + sequence_length]  # Size 1
  # Map pitches of sequence_in to integers
  network_input.append([note_to_int[char] for char in sequence_in])
  # Map integer of sequence_out to an integer
  network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [18]:
network_input.shape

(11794, 100, 1)

In [0]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Nadam')
    return model

In [20]:
model = create_network(network_input,n_vocab)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)      

In [0]:
# In case we want to use previously trained weights
weights = ""
if(len(weights)>0): model.load_weights(weights)

In [22]:
filepath = "/content/drive/My Drive/{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,
                             save_best_only=True,mode='min')

callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=75, batch_size=64, 
          callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/75
11794/11794 [==============================] - 137s 12ms/step - loss: 4.3479
Epoch 2/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1895
Epoch 3/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1795
Epoch 4/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1737
Epoch 5/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1719
Epoch 6/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1674
Epoch 7/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1710
Epoch 8/75
11794/11794 [==============================] - 132s 11ms/step - loss: 4.1662
Epoch 9/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1627
Epoch 10/75
11794/11794 [==============================] - 133s 11ms/step - loss: 4.1629
Epoch 11/75
11794/11794 [==============================] - 132s 11ms/st

In [0]:
# Generate network input again
network_input = []
output = []
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

In [24]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for i,note_index in enumerate(range(500)):
  prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
  prediction_input = prediction_input / float(n_vocab)
  prediction = model.predict(prediction_input, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  prediction_output.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

 Predicted  499   D2

In [0]:
from music21 import stream

In [27]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5
    
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/drive/My Drive/battles_nadam_output.mid')

'/content/drive/My Drive/battles_nadam_output.mid'